In [ ]:
import torch
import numpy as np
import trimesh
from matplotlib import pyplot as plt
import json
from data.garment_dataset import GarmentDataset
from utils.helpers import *
from data.normalization import get_normalized_body
from tqdm.auto import tqdm

device = "cuda"
torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# MGDDG preparation
This can be used as reference purpose to prepare our data for Motion-Guided-Deep-Dynamic-3D-Garment

In [ ]:
with open("/home/adumouli/Bureau/garment-diffusion/configs/config.json") as f:
    config = json.load(f)
dataset = GarmentDataset(config, device="cuda:0", subsets=['TEST'])

body_model = dataset.body_model
template = dataset.template

In [ ]:
# export body UV

data = dataset[0]
data['betas'][:] = 0
data['poses'][:] = 0
data['trans'][:] = 0
data['poses'][..., 0] = torch.pi * 0.5
f = body_model.get_faces().cpu()
v = body_model(data).cpu()[0]
mesh = trimesh.Trimesh(v.cpu(), f.cpu())
mesh.vertex_normals
mesh.export("/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/uv/uv_body/body_geo.ply", encoding='ascii', include_attributes=True)
mesh.export("/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/uv/uv_body/body_geo.obj")

mesh.show()

In [ ]:
path = "/home/adumouli/Data/BODY/smpl_uv_20200910/smpl_uv.obj"
from pytorch3d.io import load_obj
v, f, p = load_obj(path, load_textures=False)
# f = f.verts_idx,
(v,) = p.verts_uvs,
v = torch.cat((v, torch.zeros(v.shape[0], 1)), dim=-1)
f = f.textures_idx
print(v.shape)
print(f.shape)

mesh = trimesh.Trimesh(v, f)
mesh.show()
mesh.export("/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/uv/uv_body/body_uv.obj")

In [ ]:
# export cloth UV
v = dataset.template.vt.cpu()
f = dataset.template.fvt.cpu()
v = torch.cat((v, torch.zeros(v.shape[0], 1)), dim=-1)

print(v.shape)
print(f.shape)
mesh = trimesh.Trimesh(v.cpu(), f.cpu(), vertex_colors=(192,192,192,255))
mesh.show()
mesh.export("/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/uv/uv_tshirt/garment_uv.ply", encoding='ascii', include_attributes=True)

v = dataset.template.v.cpu()
f = dataset.template.f.cpu()
print(v.shape)
print(f.shape)
mesh = trimesh.Trimesh(v.cpu(), f.cpu(), vertex_colors=(192,192,192,255))
mesh.show()
mesh.export("/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/uv/uv_tshirt/garment_geo.ply", encoding='ascii', include_attributes=True)


# Prepare training and testing

In [ ]:
with open("/home/adumouli/Bureau/garment-diffusion/configs/config.json") as f:
    config = json.load(f)
print(config)
config['model']['sequence_length'] = 1
dataset = GarmentDataset(config, device="cuda:0", subsets=['TEST'])

body_model = dataset.body_model
template = dataset.template
body_f = body_model.get_faces().cpu()
cloth_f = template['f'].cpu()

In [ ]:
# test set
seq_dict = dataset.get_seq_dict()
root = "/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/testing_data/"
for k, seq in tqdm(seq_dict.items()):
    start = seq[0]

    seq_path = os.path.join(root, k[5:])

    body_path = os.path.join(seq_path, "body_n")
    os.makedirs(body_path, exist_ok=True)

    cloth_path = os.path.join(seq_path, "dress")
    os.makedirs(cloth_path, exist_ok=True)

    for i in range(start, start+seq[1]):
        data = dataset[i]

        trans, poses = get_normalized_body(data)
        data['trans'] = trans
        data['poses'] = poses
        
        cloth_mesh = trimesh.Trimesh(data.cloth_vertices.cpu(), cloth_f)
        cloth_mesh.vertex_normals
        out_path = os.path.join(cloth_path, str(i-start)+".obj")
        cloth_mesh.export(out_path)

        body_v = body_model(data)[0]
        body_mesh = trimesh.Trimesh(body_v.cpu(), body_f)
        body_mesh.vertex_normals
        out_path = os.path.join(body_path, str(i-start).zfill(6)+".obj")
        body_mesh.export(out_path)
        

In [ ]:
with open("/home/adumouli/Bureau/garment-diffusion/configs/config.json") as f:
    config = json.load(f)
print(config)
config['model']['sequence_length'] = 1
dataset = GarmentDataset(config, device="cuda:0")

body_model = dataset.body_model
template = dataset.template
body_f = body_model.get_faces().cpu()
cloth_f = template['f'].cpu()

In [ ]:
import re
seq_dict = dataset.get_seq_dict()
print(seq_dict)
for k in seq_dict:
    if re.search('C4-Runtowalk1', k) is None:
        continue
    print(k)

In [ ]:
# train set
seq_dict = dataset.get_seq_dict()
root = "/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/training_data/"
for k, seq in tqdm(seq_dict.items()):
    if re.search('C4-Runtowalk1', k) is None:
        continue
    start = seq[0]

    seq_path = os.path.join(root, k[6:])

    body_path = os.path.join(seq_path, "body_n")
    os.makedirs(body_path, exist_ok=True)

    cloth_path = os.path.join(seq_path, "dress")
    os.makedirs(cloth_path, exist_ok=True)

    for i in range(start, start+seq[1]):
        data = dataset[i]

        trans, poses = get_normalized_body(data)
        data['trans'] = trans
        data['poses'] = poses
        
        # cloth_mesh = trimesh.Trimesh(data.cloth_vertices.cpu(), cloth_f)
        # cloth_mesh.vertex_normals
        # out_path = os.path.join(cloth_path, str(i-start)+".obj")
        # cloth_mesh.export(out_path)

        body_v = body_model(data)[0]
        body_mesh = trimesh.Trimesh(body_v.cpu(), body_f)
        body_mesh.vertex_normals
        out_path = os.path.join(body_path, str(i-start).zfill(6)+".obj")
        body_mesh.export(out_path)
        

In [ ]:
# export body UV

data = dataset[0]
data['betas'][:] = 0
data['poses'][:] = 0
data['trans'][:] = 0
data['poses'][..., 0] = torch.pi * 0.5
f = body_model.get_faces().cpu()
v, j = body_model(data, Jtr=True)

In [ ]:
torch.save(j[0,0], "/home/adumouli/Dev/EXPERIMENT/Motion-Guided-Deep-Dynamic-3D-Garment/training_testing_data/uv/uv_body/data_center.pt")

In [ ]:
print(j[0,0])